# Lab 4: Divide and Conquer Algorithms

> Date: 11/08/2024
> Author: Adam Haile, Leigh Goetsch, Aiden Miller
> Class: CSC3310
>
> ### Learning Outcomes
>
> - Design an algorithm for a given computational problem statement
> - Justify the correctness of an algorithm
> - Perform asymptotic complexity analysis of the run time of an algorithm
> - Generate test cases for an algorithm
> - Correctly implement an algorithm from pseudocode
> - Design and execute benchmarks for an algorithm

